In [ ]:
# SVMD test using signal 1 from the paper:
# 'A Sequential Variational Mode Decomposition Method'.

# Import modules
import numpy as np
import sys
sys.path.insert(0, '..')
from modules.tfa import rfft_mf, hilbert_af
from modules.tfa_svmd import svmd, svmd_refined
from modules.plot import plot, subplots, plot_modes

In [ ]:
# Define signal
sr, du, du_extend = 5000, 1.0, 0.2
y_size = int(sr*(du+2*du_extend))
t = np.arange(y_size)/sr - du_extend
y1 = 2*t
y2 = 0.5*np.exp(-5*np.square(t-0.5))*np.sin(200*np.pi*t)
y3 = np.sin(100*np.pi*t-10*np.pi*np.square(t))
y4 = np.random.normal(scale=0.1, size=t.size)
y = np.sum([y1,y2,y3,y4], axis=0)
#y = np.load('signal 1 extended.npy')
#np.save('signal 1 extended.npy', y)

In [ ]:
# Set parameters
nmode = 3
merge_range = 1.5
out_thr, in_thr = 1e-5, 1e-10
out_iter_max, in_iter_max = 3, 150
alpha, beta = 1., 0.75e-2

In [ ]:
# Decompose
y_rfft, y_rfftf = rfft_mf(y, sr)
Modes, res = svmd(y, sr, out_thr, in_thr, out_iter_max, in_iter_max, alpha, beta)
#Modes, res = svmd_refined(y, sr, nmode, merge_range, out_thr, in_thr, out_iter_max, in_iter_max, alpha, beta)
res_rfft, res_rfftf = rfft_mf(res, sr)
Modes_rfft, Modes_rfftf = rfft_mf(Modes, sr, axis=1)
am, fm = hilbert_af(Modes, sr, axis=1)

In [ ]:
# Plot
%matplotlib ipympl
plot(y_rfft, y_rfftf, title='original signal spectrum', xlabel='frequency', ylabel='magnitude')
plot(res_rfft, res_rfftf, title='SVMD residual spectrum', xlabel='frequency', ylabel='magnitude')

plot_modes(np.array([y1, y2, y3]), t, y, y4, title='original signal and modes')
plot_modes(Modes, t, y, res, title='SVMD decomposed modes')

subplots(Modes_rfft, Modes_rfftf, title='SVMD modes rfft', subtitle='mode', \
         xlabel='frequency', ylabel='magnitude')
subplots(am, t, title='SVMD instantaneous amplitude (envelope)', subtitle='mode', \
         xlabel='time', ylabel='amplitude')
subplots(fm, t[:-1], title='SVMD instantaneous frequency', subtitle='mode', \
         xlabel='time', ylabel='frequency')